# **Library Book Recommendation System**
---
Authors: [Monicah Iwagit](https://github.com/Okodoimonicah), [Bradley Azegele](https://github.com/Azegele), [Emmanuel Kipkorir](), [Belinda Nyamai](), [Femi Kamau](https://github.com/ctrl-Karugu), and [Dennis Kimiri]()

## 1. Business Understanding

### 1.1 Problem Statement
>The tremendous growth and usage of information has led to information overloading where users find it difficult to locate the right information at a specified time. Although there are previous studies conducted on library recommender systems, the datasets used were small compared to the dataset we intend to use hence minimal area coverage. Recommending the right library books is a challenge due to the variety of genres available and the huge collection of books provided. A user finds it difficult to select the most appropriate book that will suit their academic needs, this process consumes a lot of time that the user would have used to sharpen on their desired skills. Additionally, many books in the library are rarely utilized which results in a waste of library resources. Having a personalized recommendation system seeks to predict the preference based on the user’s interest, behavior and information. The application of recommender systems in a library set up solves the problem of difficulty in choosing books and improves utilization rate of library resources. 

## 2. Data Understanding
> The goal of the Data Understanding phase is to provide a solid foundation for the subsequent steps, including data preparation, exploratory data analysis, model development, and evaluation.

#### Load Libraries

In [1]:
# Data Manipulation
import pandas as pd
import numpy as np
import re
from string import punctuation
import isbnlib

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous
import warnings

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

### 2.1 Collect Initial Data
> The aim of this section is to acquire the necessary data and load it into the notebook.

The data used in this project was extracted from 2 different sources:
>   1. The first was source was from Cai-Nicolas Ziegler in a 4-week crawl (August-September [2004]) in the University of Freiburg, Germany. This can be found [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). The data obtained are 3 datasets with information on [users](./data/BX-Users.csv), [ratings](./data/BX-Book-Ratings.csv) and [books](./data/BX-Books.csv) from a library.
>
>   2. The second set of data was sourced from the [Google Books API](https://developers.google.com/books). This data was obtained by using ISBNs in the books dataset from the first source. The python script used to extract the data can be found [here](./make_dataset.py). This data was then stored in csv format and can be found [here](./data/api-data.csv).

#### Load Data

In [2]:
# (Source 1)
ratings = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", on_bad_lines="skip", encoding="iso-8859-1")
books = pd.read_csv("data/BX-Books.csv", sep=";", on_bad_lines="skip", encoding="iso-8859-1")
users = pd.read_csv("data/BX-Users.csv", sep=";", on_bad_lines="skip", encoding="iso-8859-1")

# (Source 2)
books_extra = pd.read_csv("data/api-books.csv", dtype={"join_isbn_10": str})

### 2.2 Describe Data
> The aim of this section is to examine the data and document its surface properties.

*(Needs a initial description of the surface properties that we shall be looking at in this section)*

In [3]:
# Define class to describe the data 
class DescribeData:
    """This class is used to describe the data in a DataFrame."""
    
    def __init__(self, df) -> None:
        
        # Get the DataFrame
        self.df = df
        
        # Get the datatypes of the columns in the DataFrame
        self.datatypes = df.dtypes
         
        # Get the shape of the DataFrame
        self.shape = f"The shape of the DataFrame is {df.shape}\n\nRecords: {df.shape[0]}\nColumns: {df.shape[1]}"
        
        # Get the number of missing values in each column
        self.missing = f"There are a total of {df.isnull().sum().values.sum()} missing values in the DataFrame.\n\n{df.isnull().sum()}"
        
        # Get the number of duplicate rows
        self.duplicates = f"There are {df.duplicated().sum()} duplicated records in the DataFrame."

#### 2.2.1 `ratings`

In [4]:
# Create an instance of the 'DescribeData' class for the 'ratings' DataFrame
rt = DescribeData(ratings)

In [5]:
# Preview the 'ratings' DataFrame
rt.df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
# Inspect the information about the DataFrame
rt.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [7]:
# Inspect the shape of the DataFrame
print(rt.shape)

The shape of the DataFrame is (1149780, 3)

Records: 1149780
Columns: 3


In [8]:
# Inspect the datatypes of the columns in the DataFrame
print(rt.datatypes)

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object


In [9]:
# Inspect the summary statistics of the DataFrame
rt.df.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [10]:
# Inspect the number of missing values in each column
print(rt.missing)

There are a total of 0 missing values in the DataFrame.

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [11]:
# Inspect the duplicates in the DataFrame
print(rt.duplicates)

There are 0 duplicated records in the DataFrame.


[**Ratings Dataset**](./data/BX-Book-Ratings.csv)

>* Spans **3** columns and **1,149,780** rows.

>* Contains **2** unique datatypes: 
>    * **int64**
>    * **object**

>* No missing values

>* No duplicates

#### 2.2.2 `books` 

In [12]:
# Instantiate the 'DescribeData' class for the 'books' DataFrame
bk = DescribeData(books)

In [13]:
# Preview the 'books' DataFrame
bk.df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [14]:
# Inspect the information about the DataFrame
bk.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [15]:
# Inspect the shape of the DataFrame
print(bk.shape)

The shape of the DataFrame is (271360, 8)

Records: 271360
Columns: 8


In [16]:
# Inspect the datatypes of the columns in the DataFrame
print(bk.datatypes)

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object


In [17]:
# Inspect the missing values in the DataFrame
print(bk.missing)

There are a total of 6 missing values in the DataFrame.

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [18]:
# Inspect the duplicates in the DataFrame
print(bk.duplicates)

There are 0 duplicated records in the DataFrame.


[**Books Dataset**](./data/BX-Books.csv)

>* Spans **8** columns and **271,360** rows.

>* Contains 1 unique datatype: 
>   * **object**

>* **6** missing values:
>   * Book-Author: **1** 
>   * Publisher: **2**
>   * Image-URL-L: **3**

>* No duplicates

#### 2.2.3 `users` 

In [19]:
# Instantiate the 'DescribeData' class for the 'users' DataFrame
ur = DescribeData(users)

In [20]:
# Preview the 'users' DataFrame
ur.df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [21]:
# Inspect the information about the DataFrame
ur.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [22]:
# Inspect the shape of the DataFrame
print(ur.shape)

The shape of the DataFrame is (278858, 3)

Records: 278858
Columns: 3


In [23]:
# Inspect the datatypes of the columns in the DataFrame
print(ur.datatypes)

User-ID       int64
Location     object
Age         float64
dtype: object


In [24]:
# Inspect the summary statistics of the DataFrame
ur.df.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [25]:
# Inspect the missing values in the DataFrame
print(ur.missing)

There are a total of 110762 missing values in the DataFrame.

User-ID          0
Location         0
Age         110762
dtype: int64


In [26]:
# Inspect the duplicates in the DataFrame
print(ur.duplicates)

There are 0 duplicated records in the DataFrame.


[**Users Dataset**](./data/BX-Users.csv)

>* Spans **3** columns and **278,858** rows.

>* Contains **3** unique datatypes:
>   * **int64**
>   * **float64**
>   * **object**

>* **110,762** missing values:
>   * Age: **110,762**

>* No duplicates

#### 2.2.4 `books_extra`

In [27]:
# Instantiate the 'DescribeData' class for the 'books_extra' DataFrame
bk_extra = DescribeData(books_extra)

In [28]:
# Preview the 'books_extra' DataFrame
bk_extra.df.head()

,authors,published_date,description,isbn_10,isbn_13,page_count,categories,maturity_rating,language,join_isbn_10
0,"['Mark P. O. Morford', 'Robert J. Lenardon']",2003,Provides an introduction to classical myths pl...,0195153448,9780195153446.0,808.0,['Social Science'],NOT_MATURE,en,0195153448
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002005018
2,"[""Carlo D'Este""]",1991,"Here, for the first time in paperback, is an o...",IND:30000026059836,NaN,582.0,['1940-1949'],NOT_MATURE,en,0060973129
3,['Gina Bari Kolata'],1999,"Describes the great flu epidemic of 1918, an o...",9780374157067,374157065.0,367.0,['Medical'],NOT_MATURE,en,0374157065
4,['E. J. W. Barber'],1999,A look at the incredibly well-preserved ancien...,0393045218,9780393045215.0,240.0,['Design'],NOT_MATURE,en,0393045218


In [29]:
# Inspect the information about the DataFrame
bk_extra.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271044 entries, 0 to 271043
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   authors          210684 non-null  object 
 1   published_date   212589 non-null  object 
 2   description      180298 non-null  object 
 3   isbn_10          212653 non-null  object 
 4   isbn_13          179739 non-null  object 
 5   page_count       211385 non-null  float64
 6   categories       199011 non-null  object 
 7   maturity_rating  212741 non-null  object 
 8   language         212741 non-null  object 
 9   join_isbn_10     271044 non-null  object 
dtypes: float64(1), object(9)
memory usage: 20.7+ MB


In [30]:
# Inspect the shape of the DataFrame
print(bk_extra.shape)

The shape of the DataFrame is (271044, 10)

Records: 271044
Columns: 10


In [31]:
# Inspect the datatypes of the columns in the DataFrame
print(bk_extra.datatypes)

authors             object
published_date      object
description         object
isbn_10             object
isbn_13             object
page_count         float64
categories          object
maturity_rating     object
language            object
join_isbn_10        object
dtype: object


In [32]:
# Inspect the summary statistics of the DataFrame
bk_extra.df.describe()

,page_count
count,211385.000000
mean,279.534995
std,179.001719
min,0.000000
25%,176.000000
50%,260.000000
75%,356.000000
max,3596.000000


In [33]:
# Inspect the missing values in the DataFrame
print(bk_extra.missing)

There are a total of 607555 missing values in the DataFrame.

authors            60360
published_date     58455
description        90746
isbn_10            58391
isbn_13            91305
page_count         59659
categories         72033
maturity_rating    58303
language           58303
join_isbn_10           0
dtype: int64


In [34]:
# Inspect the duplicates in the DataFrame
print(bk_extra.duplicates)

There are 0 duplicated records in the DataFrame.


[**Books (Extra) Dataset**](./data/api-books.csv)

>* Spans **10** columns and **271,044** rows.

>* Contains **2** unique datatypes:
>   * **float64**
>   * **object**

>* **607,555** missing values:
>   * authors: **60,360**
>   * published_date: **58,455**
>   * description: **90,746**
>   * isbn_10: **58,391**
>   * isbn_13: **91,305**
>   * page_count: **59,659**
>   * categories: **72,033**
>   * maturity_rating: **58,303**
>   * language: **58,303**

>* No duplicates

## 3. Data Preparation
> The goal of the Data Preparation phase is to prepare the final data set(s) for modeling. It has five tasks:

>   * Select Data
>   * Clean Data
>   * Construct Data
>   * Integrate Data
>   * Format Data

### 3.1 Select Data
> The aim of this section is to determine which datasets will be used and document reasons for inclusion/exclusion.

### 3.2 Clean Data
> The aim of this section is to identify and correct (or remove) corrupt or inaccurate records from the datasets. 

In [35]:
# Define a superclass to clean the data
class DataCleaning:
    def __init__(self, data):
        self.data = data

    def drop_null_rows(self):
        """Drops rows with null values."""
        self.data = self.data.dropna()
        return self.data

    def drop_duplicate_rows(self):
        """Drops duplicate rows."""
        self.data = self.data.drop_duplicates()
        return self.data

    def handle_missing_values(self, column, method, value=None):
        """Handles missing values in a specified column.

        The method parameter take one of the following:
        "drop": Drops rows with missing values in the specified column.
        "fill": Fills missing values with the specified value.
        "interpolate": Interpolates missing values using linear interpolation.
        """
        if method == "drop":
            self.data = self.data.dropna(subset=[column])
        elif method == "fill":
            self.data[column] = self.data[column].fillna(value)
        elif method == "interpolate":
            self.data[column] = self.data[column].interpolate()
        else:
            raise ValueError(
                "Invalid value for method parameter. Valid values are 'drop', 'fill', and 'interpolate'."
            )
        return self.data

    def convert_column_data_type(self, column, data_type):
        """Converts the data type of a specified column."""
        self.data[column] = self.data[column].astype(data_type)
        return self.data

#### 3.2.1 [`books`](./data/BX-Books.csv)
As seen earlier, the [`books`](./data/BX-Books.csv) dataset contains **271,360** rows and **8** columns. The columns are:

>   * **ISBN**: International Standard Book Number (ISBN 10)
>   * **Book-Title**: Title of the book
>   * **Book-Author**: Author of the book
>   * **Year-Of-Publication**: Year the book was published
>   * **Publisher**: Publisher of the book
>   * **Image-URL-S**: Small image of the book
>   * **Image-URL-M**: Medium image of the book
>   * **Image-URL-L**: Large image of the book

The cleaning process for the dataset will revolve around the following questions:

1. How will we handle the 6 missing values (that we identified earlier)?
2. Though there aren't any duplicate records, do the individual columns have any duplicates? Are these duplicates erroneous?
3. Is there any invalid data?

##### 3.2.1.1 `ISBN`
> [**ISBN**](https://isbn-information.com/the-10-digit-isbn.html) (International Standard Book Number), is a unique numeric commercial book identifier. It is 10 or 13 digits long. The ISBN-10 is 10 digits long and is made up of 9 digits plus a check digit (which may be 'X') and the ISBN-13 is 13 digits long and is made up of 12 digits plus a check digit.

Cleaning the `ISBN` column will involve the following steps:
* Check whether we are dealing with ISBN-10 or ISBN-13
* Check for and inspect any invalid ISBNs
* Check for and inspect any duplicate ISBNs

**Note**:

> We will be using the [Google Books API](https://developers.google.com/books) if we need to manually cross-check the validity of the ISBNs in this dataset. In order to check an ISBN, append the ISBN-10 of the book to the end of the following url:
> 
> > `https://www.googleapis.com/books/v1/volumes?q=isbn:`
> 
>For example, to check the validity of the ISBN-10 `0441172717`, you would append it to the url as follows: 
>
>> `https://www.googleapis.com/books/v1/volumes?q=isbn:0441172717`
>
>If the ISBN is valid, the API will return a JSON object with the book's information. If the ISBN is invalid, the API will return an empty JSON object. 

In [47]:
# Create a subclass that will be used to clean the ISBN column
class CleanISBN:
    """This class is used to inspect and clean the ISBN column."""

    def __init__(self, df, col):

        # Get the DataFrame
        self.df = df

        # Get the column
        self.column = df[col]

        # Display distribution of the lengths of the values in the column
        self.length = df[col].str.len().value_counts()

    # --- FUNCTIONS ---

    # Define a function to convert the ASINs to their respective ISBN-10s
    def convert_asin(isbn):
        """This function is used to convert the ASINs to ISBN-10."""
        pass
    
    # Define a function to correct the ISBNs with invalid lengths
    def correct_length(isbn):
        """This function is used to correct the isbns containing invalid lengths."""
        
        isbn = str(isbn)
        
        # Correction of ISBNs that are less than 10 characters long
        if len(isbn) < 10:
            
            # Calculate the number of "0"s that are missing
            missing = (10 - len(isbn)) * "0"
            
            # Prepend the missing characters to the ISBN
            isbn = missing + isbn
            
            return isbn
        
        # Correction of ISBNs that are more than 10 characters long
        if len(isbn) > 10:
            
            # Try to extract the first 10 characters that are either numeric or "X"
            try:
                pattern = r"^[0-9X]{10}"
                isbn = re.search(pattern, isbn).group(0)
                
                return isbn
            
            # If the ISBN does not contain enough numeric characters, return the original ISBN
            except:
                return isbn

    # --- METHODS ---

    # Define a method to inspect and clean the invalid characters
    def invalid_char(self, action: str, check: str = "invalid"):
        """This method is used to inspect or clean the invalid characters."""

        if action == "inspect":

            if check == "invalid":
                # Query the records that have an ISBN with any non-numeric characters (except 'X')
                invalid_char = self.df.query("ISBN.str.contains('[^\dX]')")

            if check == "lowercase":
                # Query the records that contain lowercase characters
                invalid_char = self.df.query("ISBN.str.contains('[a-z]')")

            if check == "alphabetical":
                # Query the records that contain alphabetical characters (except 'X')
                invalid_char = self.df.query("ISBN.str.contains('[A-WY-Za-wy-z]')")

            if check == "non-alphanumeric":
                # Query the records that contain non-alphanumeric characters
                invalid_char = self.df.query("ISBN.str.contains('[\W_]')")

            # Print the number of records with invalid characters in the ISBN column
            print(
                f"There are {invalid_char.shape[0]} records with {check} characters in their ISBNs."
            )

            # Preview the invalid ISBN records
            return invalid_char.iloc[:, 0:5].head()

        if action == "clean":

            # Convert the ASINs to their respective ISBN-10s
            # self.column = self.column.apply(self.convert)

            # Convert the ISBNs to uppercase
            self.column = self.column.str.upper()

            # Replace the non-alphanumeric characters with empty strings
            self.column = self.column.str.replace("[\W_]", "")

            return self.column

    # Define a method to inspect and clean the invalid lengths
    def invalid_length(self, action: str, check: str = "invalid"):
        """This method is used to inspect or clean the invalid lengths."""

        if action == "inspect":

            if check == "invalid":
                # Query the records that have an ISBN with a length other than 10
                invalid_length = self.df.query("ISBN.str.len() != 10")

            if check == "less":
                # Query the records that have an ISBN with a length less than 10
                invalid_length = self.df.query("ISBN.str.len() < 10")

            if check == "greater":
                # Query the records that have an ISBN with a length greater than 10
                invalid_length = self.df.query("ISBN.str.len() > 10")

            # Print the number of records with invalid lengths in the ISBN column
            print(f"There are {invalid_length.shape[0]} records with ISBN lengths that are {check}.")

            # Preview the invalid ISBN records
            return invalid_length.iloc[:, 0:5].head()

        if action == "clean":
            
            # Apply the 'correct_length' function to the ISBN column
            self.column = self.column.apply(self.correct_length)
            
            return self.column

    # Define a method to inspect and clean the invalid check digits
    def invalid_check_digit(self, action: str):
        """This method is used to inspect or clean the invalid check digits."""

        if action == "inspect":
            pass

        if action == "clean":
            pass


# Instantiate the 'CleanISBN' class
clean_isbn = CleanISBN(books, "ISBN")

**Question 1**: *Are we dealing with ISBN-10 or ISBN-13?*

In [37]:
# Inspect the distribution of the lengths of the individual values in the 'ISBN' column
clean_isbn.length

10    271356
13         3
11         1
Name: ISBN, dtype: int64

Majority of the values in the `ISBN` column seem to be of the **ISBN-10** format. It is also worth noting that there are some 3 values that seem to be of the ISBN-13 format, and 1 value that has 11 characters. These will be investigated further as we proceed. 

**Question 2**: *Are there any invalid ISBNs?*

We have confirmed that we are dealing with ISBN-10s. Therefore, we will check for invalid ISBN-10 values. This process will involve checking for the following:
> * Invalid characters (i.e. non-numeric characters other than 'X')
> * Invalid length (i.e. length other than 10)
> * Invalid check digit (i.e. the last digit does not mathematically add up)

We begin by checking for invalid characters. This process will mainly utilize the pandas string method [`.contains()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html#pandas.Series.str.contains) coupled with regular expressions to identify the invalid characters. 

In [38]:
# Inspect the column for records with invalid characters in their ISBNs
clean_isbn.invalid_char(action='inspect', check='invalid')

There are 529 records with invalid characters in their ISBNs.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
5726,096788330x,Duncan Delaney and the Cadillac of Doom,A. L. Haskett,2000,Jonlin Books
6291,B00009ANY9,Cane River,Lalita Tademy,2001,Warner Books
6635,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
9907,039330678x,Every Person's Life Is Worth a Novel,Erving Polster,1990,W W Norton &amp; Co Inc
10272,B0000A2U93,Carmilla,Joseph Sheridan Le Fanu,0,Soft Editions Ltd


These **529** invalid ISBNs could be caused by the following:
* **Lowercase characters** (i.e. 'x' instead of 'X')
* **Alphabetical characters** (i.e. 'A', 'B', 'C', etc. except 'X')
* **Non-alphanumeric characters** (i.e. '!', '@', ' ', etc.)

We will be inspecting the ISBNs to ascertain the cause of the invalid characters.

In [39]:
# Query the records for lowercase characters in the ISBNs
clean_isbn.invalid_char(action='inspect', check='lowercase')

There are 411 records with lowercase characters in their ISBNs.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
5726,096788330x,Duncan Delaney and the Cadillac of Doom,A. L. Haskett,2000,Jonlin Books
6635,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
9907,039330678x,Every Person's Life Is Worth a Novel,Erving Polster,1990,W W Norton &amp; Co Inc
11925,014062080x,The Scarlet Letter (Penguin Popular Classics),Nathaniel Hawthorne,1994,Penguin Books Ltd
14621,042519065x,Wings of Fire,Dale Brown,2003,Berkley Publishing Group


There are **411** ISBNs that contain lowercase characters. These characters will be replaced with their uppercase counterparts.

In [40]:
# Query the records containing alphabetical characters except 'X' in the ISBNs
clean_isbn.invalid_char(action='inspect', check='alphabetical')

There are 114 records with alphabetical characters in their ISBNs.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
6291,B00009ANY9,Cane River,Lalita Tademy,2001,Warner Books
10272,B0000A2U93,Carmilla,Joseph Sheridan Le Fanu,0,Soft Editions Ltd
11185,B0000633PU,The Story of Aladdin and the Wonderful Lamp,S. Lane Poole,0,Renaissance eBooks
13862,B00007FYKO,Bloodcurdling Tales of Horror and the Macabre:...,H. P. Lovecraft,0,Ballantine
13865,B00009APKU,Moby Dick,Herman Melville,0,"Outrigger Publishing, LLC"


There are **118** ISBNs that contain alphabetical characters. Upon doing research, it turns out that some of these values are valid. However, they are not in the ISBN-10 format. They are instead using Amazon's proprietary ASIN (Amazon Standard Identification Number).

> [**ASIN**](https://www.nchannel.com/blog/amazon-asin-what-is-an-asin-number/) (Amazon Standard Identification Number) is a 10-character alphanumeric unique identifier assigned by Amazon.com and its partners for product identification within the Amazon organization. ASINs are used to identify products in Amazon listings, product advertising API, etc.

As these are not too many ISBNs, we will attempt to correct them by first checking if the books already exist in the dataset. If they do no, we will replace the ASIN with the correct record's ISBN. If they don't, we will attempt to manually search the web for and update the ISBNs.

In [41]:
# Query the records containing non-alphanumeric characters in the ISBNs
clean_isbn.invalid_char(action='inspect', check='non-alphanumeric')

There are 4 records with non-alphanumeric characters in their ISBNs.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
111808,0486404242\t,War in Kind: And Other Poems (Dover Thrift Edi...,Stephen Crane,1998,Dover Publications
171206,3518365479<90,"Suhrkamp TaschenbÃ?Â¼cher, Nr.47, Frost",Thomas Bernhard,1972,Suhrkamp
251424,3442248027 3,Diamond Age. Die Grenzwelt.,Neal Stephenson,2000,Goldmann
251649,0385722206 0,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE,2002,Anchor


From the preview above, we see that there are 4 records containing non-alphanumeric (Neither alphabetical or numerical) characters. Therefore, to correct these invalid characters, we will simply replace them with empty strings.

In [42]:
# Clean the invalid characters in the ISBNs
books['ISBN'] = clean_isbn.invalid_char(action='clean')

When it comes to invalid length, we had earlier identified 4 ISBNs that were not 10 characters/digits long. In addition, after removing any invalid characters, this number may have changed.

In [43]:
# Query the records with invalid ISBN lengths
clean_isbn.invalid_length(action='inspect', check='invalid')

There are 3 records with ISBN lengths that are invalid.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
171206,351836547990,"Suhrkamp TaschenbÃ?Â¼cher, Nr.47, Frost",Thomas Bernhard,1972,Suhrkamp
251424,34422480273,Diamond Age. Die Grenzwelt.,Neal Stephenson,2000,Goldmann
251649,03857222060,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE,2002,Anchor


There are a total of 3 records with invalid ISBN lengths. When it comes to correcting ISBNs of invalid lengths, there are 2 main approaches:

1. ISBNs less than 10 characters long can be padded with leading zeros. This may have resulted from being saved in a numerical format which automatically removes leading zeros

> Example:
>
> An ISBN of the form `123456789` can be padded with leading zeros to become `0123456789`.

1. ISBNs greater than 10 characters long can be truncated to 10 characters long.

> Example:
>
> An ISBN of the form `1234567890123` can be truncated to become `1234567890`.


In [44]:
# Query the records with ISBN lengths that are less than 10 characters
clean_isbn.invalid_length(action='inspect', check='less')

There are 0 records with ISBN lengths that are less.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher


There aren't any records that have ISBNs that are less than 10 characters

In [45]:
# Query the records with ISBN lengths that are greater than 10 characters
clean_isbn.invalid_length(action='inspect', check='greater')

There are 3 records with ISBN lengths that are greater.


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
171206,351836547990,"Suhrkamp TaschenbÃ?Â¼cher, Nr.47, Frost",Thomas Bernhard,1972,Suhrkamp
251424,34422480273,Diamond Age. Die Grenzwelt.,Neal Stephenson,2000,Goldmann
251649,03857222060,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE,2002,Anchor


There are 3 records with ISBNs that are greater than 10 characters. However, if we take the first 10 characters and cross-check them against the Google-Books API, they each return data belonging to the books that they have been assigned to in the dataset. Therefore, as we had referenced earlier, we will be truncating the ISBNs to the first 10 characters.

In [48]:
# Clean the ISBNs with invalid lengths
books['ISBN'] = clean_isbn.invalid_length(action='clean')

**Question 3**: *Are there any duplicates?*

In [ ]:
# Check for any duplicated ISBNs

##### 3.2.1.2 `Book-Title`
Cleaning the `Book-Title` column will involve the following steps:
* Check for invalid characters (probably resulting from wrong encoding) and correct them where possible

##### 3.2.1.3 `Book-Author`
Cleaning the `Book-Author` column will involve the following steps:
* Check for invalid characters (probably resulting from wrong encoding) and correct them where possible
* Check for instances of misspelled names or cases where the author's name has been written differently (e.g. 'Ann M. Martin', 'Ann Martin', 'Ann M Martin)

##### 3.2.1.3 `Year-Of-Publication`
Cleaning the `Book-Author` column will involve the following steps:
* Check for invalid characters (probably resulting from wrong encoding) and correct them where possible
* Check for instances of misspelled names or cases where the author's name has been written differently (e.g. 'Ann M. Martin', 'Ann Martin', 'Ann M Martin)

In [ ]:
# Define a function to clean the ISBNs
def clean_isbn(isbn: str) -> str:
    """This function is used to clean the 'ISBN' column."""
    
    # Remove any form of punctuation or whitespace
    isbn = isbn.translate(str.maketrans('', '', punctuation)).replace(" ", "")
    
    # Convert the ISBN to uppercase
    isbn = isbn.upper()
    
    # Remove any characters that are not digits or X
    isbn = re.sub(r"[^0-9X]", "", isbn)
    

### Books Dataset

In [ ]:
#checking the shape of the books dataset before dealing with erraneous ISBN values
books.shape

In [ ]:
#converting the books ISBN numbers to upper string and replacing the quotation marks
books['ISBN'] = books['ISBN'].str.upper().str.replace(' ', '')

In [ ]:
#checking for duplicates 
books['ISBN'].duplicated().sum()

In [ ]:
#function to correct the ISBN that are not in correct format
def correct_isbn(isbn):
    isbn = str(isbn)
    fill = "0"
    if len(isbn) < 10:
        missing = 10 - len(isbn)
        new_isbn = (fill * missing) + isbn
        return new_isbn
    elif len(isbn) > 10:
        try:
            pattern = r"^[0-9Xx]{10}"
            result = re.search(pattern, isbn)
            new_isbn = result.group(0)
            return new_isbn
        except:
            print(isbn)
            return isbn

    return isbn

In [ ]:
#applying the function to the books ISBN column
books['ISBN'] = books['ISBN'].apply(correct_isbn)

In [ ]:
#dropping duplicates
books.drop_duplicates(subset=['ISBN'], inplace=True)

In [ ]:
#checking the shape of the new dataset after dealing with duplicates
books.shape

In [ ]:
#merge books dataset to books_extra
books_data = books.merge(books_extra, right_on = 'join_isbn_10', left_on = 'ISBN', how = 'inner')

In [ ]:
#Instantiate the 'DescribeData' class for the merged dataset
bk_upgrd = DescribeData(books_data)

In [ ]:
bk_upgrd.df.head()

In [ ]:
bk_upgrd.df.info()

In [ ]:
print(bk_upgrd.missing)

In [ ]:
#Instantiate the 'datacleaning' class for the merged dataset
data_cleaning = DataCleaning(books_data)

In [ ]:
#dropping rows with null values in the description column
books_data = data_cleaning.handle_missing_values(column="description", method="drop")
books_data.head()

All entries with no description are dropped because it is a critical feature that will be used in our recommendation system.

In [ ]:
#dropping unwanted columns from the merged dataset
book_data = books_data.drop(['Image-URL-L', 'Image-URL-M','isbn_10', 'isbn_13','join_isbn_10'], axis=1)
book_data

In [ ]:
#checking for missing values after dropping unwanted columns
book_data.isna().sum()

In [ ]:
#checking the years of publication
book_data['Year-Of-Publication'].unique()

In [ ]:
book_data.loc[book_data['Year-Of-Publication'] == '0',:]

In [ ]:
##Replace values where the "Year-Of-Publication" is 0 with the corresponding value in the "published-data" column
book_data["Year-Of-Publication"] = book_data["Year-Of-Publication"].mask(book_data["Year-Of-Publication"] == '0', book_data["published_date"])

In [ ]:
#confirming the years with 0 have been replaced 
book_data.loc[book_data['Year-Of-Publication'] == '0',:]

In [ ]:
#investigating the rows having 'DK Publishing Inc' as year Of Publication
book_data.loc[book_data['Year-Of-Publication'] == 'DK Publishing Inc',:]

It can be observed that there are some incorrect entries in Year-Of-Publication field since publisher names 'DK Publishing Inc' has been incorrectly recorded as Year-Of-Publication in dataset due to some errors in csv file. 

In [ ]:
#Since bookAuthor is incorrectly loaded with publication year; making required corrections
#ISBN '0789466953'
book_data.loc[book_data.ISBN == '0789466953','Year-Of-Publication'] = 2000
book_data.loc[book_data.ISBN == '0789466953','Book-Author'] = "James Buckley"
book_data.loc[book_data.ISBN == '0789466953','Publisher'] = "DK Publishing Inc"
book_data.loc[book_data.ISBN == '0789466953','Book-Title'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

#ISBN '078946697X'
book_data.loc[book_data.ISBN == '078946697X','Year-Of-Publication'] = 2000
book_data.loc[book_data.ISBN == '078946697X','Book-Author'] = "Michael Teitelbaum"
book_data.loc[book_data.ISBN == '078946697X','Publisher'] = "DK Publishing Inc"
book_data.loc[book_data.ISBN == '078946697X','Book-Title'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"

#rechecking that the errors have been corrected
book_data.loc[(book_data.ISBN == '0789466953') | (book_data.ISBN == '078946697X'),:]

In [ ]:
#investigating the rows having 'Gallimard' as yearOfPublication
book_data.loc[book_data['Year-Of-Publication'] == 'Gallimard',:]

In [ ]:
#making required corrections for Gallimard while keeping other fields intact
book_data.loc[book_data.ISBN == '2070426769','Year-Of-Publication'] = 2003
book_data.loc[book_data.ISBN == '2070426769','Book-Author'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
book_data.loc[book_data.ISBN == '2070426769','Publisher'] = "Gallimard"
book_data.loc[book_data.ISBN == '2070426769','Book-Title'] = "Peuple du ciel, suivi de 'Les Bergers"

#rechecking that the corrections have been made
book_data.loc[book_data.ISBN == '2070426769',:]

In [ ]:
#converting year of publication to integers
book_data['Year-Of-Publication']=pd.to_numeric((book_data['Year-Of-Publication']), errors='coerce')

print(sorted(book_data['Year-Of-Publication'].unique()))

In [ ]:
#replacing years above 2004 and those with no value with the median
book_data.loc[(book_data['Year-Of-Publication'] > 2004) | (book_data['Year-Of-Publication'] == 0),'Year-Of-Publication'] = np.NAN

#replacing NaNs with median value of Year-Of-Publication
book_data['Year-Of-Publication'].fillna(round(book_data['Year-Of-Publication'].median()), inplace=True)

In [ ]:
# Instantiate the 'datacleaning' class for the 'published' DataFrame
data_cleaning = DataCleaning(book_data)

In [ ]:
book_data = data_cleaning.convert_column_data_type('Year-Of-Publication', int)
print(sorted(book_data['Year-Of-Publication'].unique())) 

In [ ]:
#dropping the published date column since it will not be required for analysis
book_data = book_data.drop(['published_date'], axis=1)

In [ ]:
book_data.isna().sum()

In [ ]:
#exploring 'publisher' column
book_data.loc[book_data.Publisher.isnull(),:]

In [ ]:
#Filling Nan of Publisher with its publisher
book_data.Publisher.fillna('Bantam', inplace=True)

In [ ]:
#exploring 'Page count' column
book_data.loc[book_data.page_count.isnull(),:]

In [ ]:
# Convert the 'page_count' to numeric data type
book_data['page_count'] = pd.to_numeric(book_data['page_count'], errors='coerce')
# Replace any NaN values with a placeholder value
book_data['page_count'].fillna(-1, inplace=True)
# Convert the 'float_column' to integer data type
book_data['page_count'] = book_data['page_count'].round().astype(int)
# preview data set
book_data.head()

In [ ]:
#inspecting page_count entries with missing data with the placeholder -1
book_data.loc[book_data['page_count'] == -1,:]

There are 490 missing values in page_count that were replaced with -1.

In [ ]:
#exploring the categories column
book_data.loc[book_data.categories.isnull(),:]

In [ ]:
#filling NaN of categories with None
book_data.categories.fillna('None',inplace=True)

In [ ]:
# In the categories column we have the categories names in a list
# Remove the brackets and quotation marks from the 'categories' column
book_data['categories'] = book_data['categories'].str.strip('[]')
book_data['categories'] = book_data['categories'].str.strip("''")
# preview data set
book_data.head()

In [ ]:
#exploring the authors column
book_data.loc[book_data.authors.isnull(),:]

In [ ]:
# defining a fuction to handle missing data in authors column
def replace_missing_values(df):
 # Create a boolean mask indicating which rows in the "authors" column have missing values
 mask = df['authors'].isnull()
 
 # Get the index values for the rows where the value in the "authors" column is missing
 indexes = df.loc[mask, 'authors'].index
 
 # Use the index values to select the corresponding values in the "book_authors" column
 # and assign these values to the "authors" column for the rows where the value is missing
 df.loc[indexes, 'authors'] = df.loc[indexes, 'Book-Author']

In [ ]:
# replacing missing authors with corresponding book-author
replace_missing_values(book_data)

In [ ]:
# rechecking for missing values in authors
book_data['authors'].isna().sum()

The category column has the categories in list which is corrected by removing the brackets and quotations

In [ ]:
# In the authors column we have the authors names in a list
# Defining a function to remove the brackets and quotation marks from the 'authors' column
def authors_split(authors):
    try:
        authors = authors\
            .replace("['", "")\
            .replace("']", "")\
            .replace("', '", ", ")\
            .replace("','", ", ")\
            .replace("' ,'", ", ")\
            .replace('["', '')\
            .replace('"]', '')\
            .replace('", "', ', ')\
            .replace('","', ', ')\
            .replace('" ,"', ', ')\
            .split(", ")
            
        return authors
    
    except:
        return []

In [ ]:
#applying the function to the authors column
book_data['authors'] = book_data['authors'].apply(authors_split)
book_data['authors'].head()

In [ ]:
#separating the different authors from the created list
book_data = book_data.explode(column='authors', ignore_index = True)
book_data.head(3)

In [ ]:
#converting values in the maturity rating column to lowercase
book_data['maturity_rating'] = book_data['maturity_rating'].str.lower()
book_data.head()

In [ ]:
#confirming that all missing values have been dealt with
book_data.isna().sum()

### Users Dataset

The location column has city, state and country separated by commas. We segregate these into 3 different columns so that we can analyse on the basis of the country of different users.

In [ ]:
#splitting a string into a list
list_ = users.Location.str.split(', ')

#empty lists to add values  
city = []
state = []
country = []
count_no_state = 0    
count_no_country = 0
#generating a for loop to add values to the empty lists
for i in range(0,len(list_)):
    #removing invalid entries in city
    if list_[i][0] == ' ' or list_[i][0] == '' or list_[i][0]=='n/a' or list_[i][0] == ',':  
        city.append('other')
    else:
        city.append(list_[i][0].lower())

    if(len(list_[i])<2):
        state.append('other')
        country.append('other')
        count_no_state += 1
        count_no_country += 1
    else:
        #removing invalid entries in state
        if list_[i][1] == ' ' or list_[i][1] == '' or list_[i][1]=='n/a' or list_[i][1] == ',':   
            state.append('other')
            count_no_state += 1            
        else:
            state.append(list_[i][1].lower())
        
        if(len(list_[i])<3):
            country.append('other')
            count_no_country += 1
        else:
            #removing invalid entries in country
            if list_[i][2] == ''or list_[i][1] == ',' or list_[i][2] == ' ' or list_[i][2] == 'n/a':
                country.append('other')
                count_no_country += 1
            else:
                country.append(list_[i][2].lower())
        
#dropping the location column from users             
users = users.drop('Location',axis=1)

#handling cases where city/state from the lists is already given
temp = []
for ent in city:
    c = ent.split('/')            
    temp.append(c[0])
    
#creating a dataframe for city, state and country
df_city = pd.DataFrame(temp,columns=['City'])
df_state = pd.DataFrame(state,columns=['State'])
df_country = pd.DataFrame(country,columns=['Country'])

#adding the new dataframes to the original users dataframe 
users = pd.concat([users, df_city], axis=1)
users = pd.concat([users, df_state], axis=1)
users = pd.concat([users, df_country], axis=1)

#printing the number of countries that do not have values 
print(count_no_country)

#printing the states which didnt have any values
print(count_no_state)     

In [ ]:
## Drop duplicate rows
users.drop_duplicates(keep='last', inplace=True)
users.reset_index(drop=True, inplace=True)

In [ ]:
#confirming that the new dataframes have been added to users
users.info()

In [ ]:
users.head()

In [ ]:
# checking for null values
users['Age'].isnull().sum()

#### Dealing with missing values in the Age column

In [ ]:
#checking the Age distribution
users.Age.hist(bins=[0, 10, 20, 30, 40, 50, 100])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

The most active users are in the age bracket between 20 to 30 years.

In [ ]:
#checking for outliers in the age column using a boxplot
sns.boxplot(y='Age', data=users)
plt.title('Outliers present in the Age column')

Looking at the distribution plot the age is skewed to the right and has outliers. Replacing the missing age values and values less than 10 and greater than 80 with the median will be the robust approach since it's not sensitive to extreme values compared to using the mean.

In [ ]:
#checking all values present in the Age column
print(sorted(list(users['Age'].unique())))

In [ ]:
#plotting a distribution plot to discover the relevant age bracket 
sns.distplot(users.Age)
plt.title('Age Distribution Plot')

Evidently, users with ages less than 10 and above 80 are not an appropriate target for our book recommendation system since we assume they will not be able to access a library or use the recommendation system.

In [ ]:
# Select rows where the age column is greater than 10 and less than 80
users_filtered = users[(users['Age'] >= 10) & (users['Age'] <= 80)]

In [ ]:
# calculating the median for the required ages
median = users_filtered['Age'].median()
median

In [ ]:
# Replace the entries less than 10 or greater than 80 with the median
users.loc[(users['Age'] < 10) | (users['Age'] > 80), 'Age'] = median

In [ ]:
# Replace missing values in the "age" column with the median age
users['Age'] = users['Age'].fillna(median)

In [ ]:
# convert the age column to int
users['Age'] = pd.to_numeric(users['Age'], downcast='integer')

In [ ]:
print(sorted(list(users['Age'].unique())))

In [ ]:
#confirming that the missing values have been dealt with
users.isna().sum()

In [ ]:
users.info()

### Ratings Dataset

In [ ]:
#cleaning the ratings ISBN column using the correct_isbn function
ratings['ISBN'] = ratings['ISBN'].apply(correct_isbn)

In [ ]:
#getting rid of the special characters present in the ISBN column
from string import punctuation
ratings['ISBN'] = ratings['ISBN'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)).replace(' ', '').upper())

In [ ]:
#ensuring the special characters have been dealt with
ratings['ISBN'] = ratings['ISBN'].apply(correct_isbn)

In [ ]:
#ensuring ratings dataset has only books in the merged dataset for books and books extra
ratings_new = ratings[ratings.ISBN.isin(book_data.ISBN)]
ratings.shape,ratings_new.shape

It can be seen that many rows having book ISBN not part of the merged dataset for books and books extra got dropped off

In [ ]:
#ensuring ratings dataset has only books in the users datset
print("Shape of dataset before dropping",ratings_new.shape)
ratings_new = ratings_new[ratings_new['User-ID'].isin(users['User-ID'])]
print("shape of dataset after dropping",ratings_new.shape)

It is evident that no new user was there in the users dataset.

In [ ]:
#defining a function for plotting a countplot
def count_plot (column, dataset, heading):
    """Visualize the countplots of various dataframes with seaborn barplot
    
    Args: 
        data: dataframe of various columns and their count returned from count_plot function
        
    Returns: 
        countplot: countplot of columns and their count
    """
    plt.figure(figsize=(15,8));
    sns.countplot(y=column, data=dataset, order=pd.value_counts(dataset[column]).iloc[0:15].index);
    plt.title(heading);

In [ ]:
#checking the distribution of ratings
count_plot('Book-Rating', ratings, 'Ratings Distribution')

The ratings are unevenly distributed since majority of the ratings are 0. The ratings are either explicit, expressed on a scale of 1-10 with higher values denoting higher appreciation, or implicit, expressed by 0. Hence the need to segragate implicit and explict ratings datasets

In [ ]:
#segregating implicit and explict ratings datasets
ratings_explicit = ratings_new[ratings['Book-Rating'] != 0]
ratings_implicit = ratings_new[ratings['Book-Rating'] == 0]

In [ ]:
#plotting the explicit ratings
count_plot('Book-Rating', ratings_explicit, 'Explicit Ratings')

It can be observed that higher ratings are most common amongst users and rating 8 has been rated the highest number of times.

In [ ]:
#finding the top 5 books that are rated most
rating_count = pd.DataFrame(ratings_explicit.groupby('ISBN')['Book-Rating'].count())
rating_count.sort_values('Book-Rating', ascending=False).head()

In [ ]:
#discovering which book is displayed by the above ISBN numbers
most_rated_books = pd.DataFrame(['0316666343', '0971880107', '0385504209', '0312195516', '0060928336'], index=np.arange(5), columns = ['ISBN'])
most_rated_books_summary = pd.merge(most_rated_books, book_data, on='ISBN')
most_rated_books_summary

Majority of the books that had a high rating are mainly of category fiction and are all novels. This shows that novels of type fiction are mainly preferred by users

In [ ]:
# Create column Rating average consisting the mean of the explicit ratings
ratings_explicit['Avg_Rating']=ratings_explicit.groupby('ISBN')['Book-Rating'].transform('mean')
# Create column Total-No-Of-Users-Rated consisting the count of explicit ratings 
ratings_explicit['Total_No_Of_Users_Rated']=ratings_explicit.groupby('ISBN')['Book-Rating'].transform('count')

In [ ]:
ratings_explicit.head()

In [ ]:
#merging explicit ratings to the users dataset and published2 dataset
Final_Dataset=users.copy()
Final_Dataset=pd.merge(Final_Dataset,ratings_explicit,on='User-ID')
Final_Dataset=pd.merge(Final_Dataset,book_data,on='ISBN')

In [ ]:
Final_Dataset.head()

In [ ]:
#Instantiate the 'DescribeData' class for the 'Final_dataset' 
fd = DescribeData(Final_Dataset)

In [ ]:
#checking for missing values in the combined dataset to be used for analysis
print(fd.missing)

### Exploratory Data Analysis

In [ ]:
#plotting the count of users per country
count_plot('Country', Final_Dataset, 'Count of library users per country')

>Majority of library users are based in USA and Canada while Singapore and Italy have the least number of library users.

In [ ]:
#plotting the most recognised book authors 
count_plot('authors', Final_Dataset, 'The most recognised Book authors')

>The most recognised book authors are Stephen King and John Grisham. On the other hand, Dan Brown and Patricia Dniels Cornwell are the least recognised book authors.

In [ ]:
#plotting the most recognised publishers
count_plot('Publisher', Final_Dataset, 'The most recognised Publishers')

>The most recognised publisher is Ballantine Books and Pocket while the least recognised publisher is Harlequin and Simon &amp Schuster.